## Installation and Imports

In [1]:
!pwd

/home/daniel/DARM/darm_mujoco/darm_training


In [2]:
# Configure env variables

# TODO: change path
import os
os.environ["DARM_MUJOCO_PATH"] = "/home/daniel/DARM/darm_mujoco"
os.getenv('DARM_MUJOCO_PATH')

'/home/daniel/DARM/darm_mujoco'

In [ ]:
# Check if GCC is installed
!gcc --version

In [ ]:
# Install GCC if absent
!sudo apt update
!sudo apt install build-essential -y

In [ ]:
# # Setup Mujoco for gym - If needed
# !apt-get install -y \
#     libgl1-mesa-dev \
#     libgl1-mesa-glx \
#     libglew-dev \
#     libosmesa6-dev \
#     software-properties-common

# !apt-get install -y patchelf

# !pip install gym

# !pip install free-mujoco-py

# import mujoco_py
# import gym

In [3]:
!pip install ray[rllib] torch
!pip install wandb
!pip install tensorflow_probability

In [4]:
%cd /home/daniel/DARM/darm_mujoco
!python setup.py install

/home/daniel/DARM/darm_mujoco
running install
/home/daniel/miniconda3/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/daniel/miniconda3/lib/python3.8/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
writing darm_gym_env.egg-info/PKG-INFO
writing dependency_links to darm_gym_env.egg-info/dependency_links.txt
writing requirements to darm_gym_env.egg-info/requires.txt
writing top-level names to darm_gym_env.egg-info/top_level.txt
reading manifest file 'darm_gym_env.egg-info/SOURCES.txt'
writing manifest file 'darm_gym_env.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/e

In [ ]:
# Check if mujoco import is successful
import mujoco

In [ ]:
# If mujoco import fails, update pandas and restart runtime
!pip install pandas -U

In [3]:
import ray
from ray.rllib.algorithms.sac import SACConfig
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

from ray import air, tune
from ray.air import session
from ray.air.integrations.wandb import setup_wandb
from ray.air.integrations.wandb import WandbLoggerCallback

import gym
from darm_gym_env import DARMSFEnv

## Register Environment with RLlib

In [4]:
# env_creator = lambda env_config: gym.make("darm/DarmSFHand-v0", render_mode=None, hand_name="hand1") # DARMSFEnv(render_mode=None, reaction_time=0.08, hand_name="hand1") # 

def make_env(env_config):
    env = gym.wrappers.TimeLimit(env=DARMSFEnv(render_mode=None, reaction_time=0.08, hand_name="hand1"), max_episode_steps=1000)
    return env
env_creator = lambda env_config: make_env(env_config) #gym.wrappers.TimeLimit(env=DARMSFEnv(render_mode=None, reaction_time=0.08, hand_name="hand1"), max_episode_steps=1000)

register_env("darm/DarmSFHand-v0", env_creator)

## Configure and Run

In [14]:
# TODO:
# change: rollout_workers
# change: gpu

config = (
    SACConfig()
    .environment(
        env="darm/DarmSFHand-v0",
        normalize_actions=True
    )
    .training(
        q_model_config={
            "fcnet_activation": "relu",
            "fcnet_hiddens": [256, 256]
        },
        policy_model_config={
            "fcnet_activation": "relu",
            "fcnet_hiddens": [256, 256]
        },
        tau=0.005,
        target_entropy="auto",
        n_step=1,  # num of SGD steps per batch of data in training step
        train_batch_size=256,
        target_network_update_freq=1,
        replay_buffer_config={"type":"MultiAgentPrioritizedReplayBuffer"},
        num_steps_sampled_before_learning_starts=10_000,
        optimization_config={
          "actor_learning_rate": 0.0003,
          "critic_learning_rate": 0.0003,
          "entropy_learning_rate": 0.0003,
        },
        clip_actions=False
    )
    .rollouts(
        num_rollout_workers=3,
        num_envs_per_worker=8,
        rollout_fragment_length=1,
        recreate_failed_workers=True,
        num_consecutive_worker_failures_tolerance=10,
        restart_failed_sub_environments=True,
        batch_mode="complete_episodes"
    )
    .resources(num_gpus=0)
    .evaluation(evaluation_interval=100) # For 1000 timesteps iter; 100 evals
    .reporting(
        min_sample_timesteps_per_iteration=1000,
        metrics_num_episodes_for_smoothing=5
    )
    .framework(framework="torch")
)

{'extra_python_environs_for_driver': {},
 'extra_python_environs_for_worker': {},
 'num_gpus': 0,
 'num_cpus_per_worker': 1,
 'num_gpus_per_worker': 0,
 '_fake_gpus': False,
 'custom_resources_per_worker': {},
 'placement_strategy': 'PACK',
 'eager_tracing': False,
 'eager_max_retraces': 20,
 'tf_session_args': {'intra_op_parallelism_threads': 2,
  'inter_op_parallelism_threads': 2,
  'gpu_options': {'allow_growth': True},
  'log_device_placement': False,
  'device_count': {'CPU': 1},
  'allow_soft_placement': True},
 'local_tf_session_args': {'intra_op_parallelism_threads': 8,
  'inter_op_parallelism_threads': 8},
 'env': 'darm/DarmSFHand-v0',
 'env_config': {},
 'observation_space': None,
 'action_space': None,
 'env_task_fn': None,
 'render_env': False,
 'clip_rewards': None,
 'normalize_actions': True,
 'clip_actions': False,
 'disable_env_checking': False,
 'num_envs_per_worker': 8,
 'sample_collector': ray.rllib.evaluation.collectors.simple_list_collector.SimpleListCollector,
 's

(raylet) [2023-02-11 19:29:50,001 E 3731 3781] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_19-15-37_042436_3307 is over 95% full, available space: 117522432; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-11 19:30:00,016 E 3731 3781] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_19-15-37_042436_3307 is over 95% full, available space: 117510144; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-11 19:30:10,035 E 3731 3781] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_19-15-37_042436_3307 is over 95% full, available space: 117452800; capacity: 31845081088. Object creation will fail if spilling is required.


In [8]:
# TODO:
# change: rollout_workers
# change: gpu
# change: tags
# change: name

wandb_init = dict(
    save_code=True,
    resume=True,
    config={
        "env": "DARMSFHand-v0",
        
        "actor_learning_rate": 0.0003,
        "critic_learning_rate": 0.0003,
        "entropy_learning_rate": 0.0003,
        "framework": "torch",
        
        "num_rollout_workers": 3,
        "num_gpu": 0,
        "metrics_num_episodes_for_smoothing": 5
    },
    tags=["single_finger", "complete_episodes"],
    notes="Fixed the env to use targets that are delta increaments from the starting state. Removed velocity penalty, and used only effort penalty",
    name="8env_cmp_ep_Test_DARMSF_DELTA_TARGET"
    # job_type=
    # monitor_gym=
)

In [9]:
%cd /home/daniel/DARM/darm-mujoco/darm_training/

[Errno 2] No such file or directory: '/home/daniel/DARM/darm-mujoco/darm_training/'
/home/daniel/DARM/darm_mujoco/darm_training


In [10]:
# TODO: 
# change: name
# change: checkpoint_freq

sync_config = tune.SyncConfig()

tuner = tune.Tuner(
    "SAC",
    run_config=air.RunConfig(
        name="8env_cmp_ep_Test_DARMSF_DELTA_TARGET",
        local_dir=f"{os.getenv('DARM_MUJOCO_PATH')}/darm_training/results",
        sync_config=sync_config,
        stop={"training_iteration": 10_000, "episode_reward_mean": 200},
        checkpoint_config=air.CheckpointConfig(
            checkpoint_at_end=True,
            checkpoint_score_attribute="episode_reward_mean",  # or leave to save last chkpts
            checkpoint_score_order="max",
            checkpoint_frequency=1,  #50,
            num_to_keep=3
        ),
        callbacks=[
                WandbLoggerCallback(project="DARM", 
                                    api_key="392c8a47eb0658eb5c71190757a69110e2140f4a",
                                    save_checkpoints=True, 
                                    **wandb_init)
            ],
        ),
    param_space=config
)

results = tuner.fit()

2023-02-11 19:15:39,278	INFO worker.py:1538 -- Started a local Ray instance.


wandb: Currently logged in as: danieladejumo. Use `wandb login --relogin` to force relogin
(SAC pid=4006) 2023-02-11 19:15:44,955	WARNING algorithm_config.py:488 -- Cannot create SACConfig from given `config_dict`! Property __stdout_file__ not supported.
(SAC pid=4006) 2023-02-11 19:15:45,346	INFO algorithm.py:501 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(raylet) [2023-02-11 19:15:49,193 E 3731 3781] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_19-15-37_042436_3307 is over 95% full, available space: 584503296; capacity: 31845081088. Object creation will fail if spilling is required.
(RolloutWorker pid=4143) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=4143)   logger.warn(
(RolloutWorker pid=4144) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarni

(RolloutWorker pid=4143) Loaded XML file successfully
(RolloutWorker pid=4144) Loaded XML file successfully
(RolloutWorker pid=4142) Loaded XML file successfully


(RolloutWorker pid=4142) Loaded XML file successfully
(RolloutWorker pid=4144) Loaded XML file successfully
(RolloutWorker pid=4143) Loaded XML file successfully
(RolloutWorker pid=4142) Loaded XML file successfully
(RolloutWorker pid=4144) Loaded XML file successfully
(RolloutWorker pid=4143) Loaded XML file successfully
(RolloutWorker pid=4144) Loaded XML file successfully
(RolloutWorker pid=4142) Loaded XML file successfully
(RolloutWorker pid=4143) Loaded XML file successfully
(RolloutWorker pid=4144) Loaded XML file successfully
(RolloutWorker pid=4142) Loaded XML file successfully
(RolloutWorker pid=4143) Loaded XML file successfully


(raylet) [2023-02-11 19:15:59,205 E 3731 3781] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_19-15-37_042436_3307 is over 95% full, available space: 577929216; capacity: 31845081088. Object creation will fail if spilling is required.


...

(raylet) [2023-02-11 19:28:09,881 E 3731 3781] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_19-15-37_042436_3307 is over 95% full, available space: 147398656; capacity: 31845081088. Object creation will fail if spilling is required.


(RolloutWorker pid=9221) Loaded XML file successfully


wandb: Adding directory to artifact (/home/daniel/DARM/darm_mujoco/darm_training/results/8env_cmp_ep_Test_DARMSF_DELTA_TARGET/SAC_darm_DarmSFHand-v0_179bb_00000_0_2023-02-11_19-15-41/checkpoint_000207)... Done. 0.1s


(RolloutWorker pid=9221) Loaded XML file successfully
(RolloutWorker pid=9221) Loaded XML file successfully


wandb: Adding directory to artifact (/home/daniel/DARM/darm_mujoco/darm_training/results/8env_cmp_ep_Test_DARMSF_DELTA_TARGET/SAC_darm_DarmSFHand-v0_179bb_00000_0_2023-02-11_19-15-41/checkpoint_000208)... Done. 0.1s
wandb: Adding directory to artifact (/home/daniel/DARM/darm_mujoco/darm_training/results/8env_cmp_ep_Test_DARMSF_DELTA_TARGET/SAC_darm_DarmSFHand-v0_179bb_00000_0_2023-02-11_19-15-41/checkpoint_000209)... Done. 0.1s
(raylet) [2023-02-11 19:28:19,887 E 3731 3781] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_19-15-37_042436_3307 is over 95% full, available space: 139325440; capacity: 31845081088. Object creation will fail if spilling is required.
wandb: Adding directory to artifact (/home/daniel/DARM/darm_mujoco/darm_training/results/8env_cmp_ep_Test_DARMSF_DELTA_TARGET/SAC_darm_DarmSFHand-v0_179bb_00000_0_2023-02-11_19-15-41/checkpoint_000210)... Done. 0.1s
wandb: Adding directory to artifact (/home/daniel/DARM/darm_mujoco/darm_training/results/8env_cmp_e

In [16]:
# Ensure wandb is sysncing to cloud
# cd to darm_training again if not

In [10]:
# TODO:
# change: experiment name

# Restore Interrupted run
tuner = tune.Tuner.restore(
    f"{os.getenv('DARM_MUJOCO_PATH')}/darm_training/results/Test_DARMSF_DELTA_TARGET",
    resume_errored=True
)
tuner

2023-02-11 17:26:13,607	INFO experiment_analysis.py:795 -- No `self.trials`. Drawing logdirs from checkpoint file. This may result in some information that is out of sync, as checkpointing is periodic.


(raylet) [2023-02-11 17:26:16,601 E 14732 14777] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_17-23-24_266872_14581 is over 95% full, available space: 1061720064; capacity: 31845081088. Object creation will fail if spilling is required.


In [15]:
results = tuner.get_results()
results

In [16]:
# Get the best result based on a particular metric.
best_result = results.get_best_result(metric="episode_reward_mean", mode="max")
best_result

Result(metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 8.343955039978027, 'actor_loss': -5.076763153076172, 'critic_loss': 0.4612053632736206, 'alpha_loss': -0.850807249546051, 'alpha_value': 0.9030595, 'log_alpha_value': -0.101966895, 'target_entropy': -5.0, 'policy_t': -0.01997794397175312, 'mean_q': 2.0334110260009766, 'max_q': 2.8355112075805664, 'min_q': 1.038293480873108}, 'td_error': array([9.46030378e-01, 4.29627061e-01, 2.65497327e-01, 8.43869328e-01,
       1.09686172e+00, 7.66791701e-01, 7.26696014e-01, 5.70532084e-02,
       1.14584994e+00, 4.43507016e-01, 1.04901314e-01, 1.52089047e+00,
       6.52013183e-01, 8.16148460e-01, 1.08409297e+00, 2.61833251e-01,
       3.79876256e-01, 9.56449747e-01, 5.75677335e-01, 1.02149987e+00,
       1.76170349e-01, 9.50863540e-01, 7.04805613e-01, 3.25276971e-01,
       6.36387825e-01, 8.46629441e-01, 5.59558868e-02, 8.83865356e-03,
       

In [17]:
# Get the best checkpoint corresponding to the best result.
best_checkpoint = best_result.checkpoint
best_checkpoint

Checkpoint(local_path=/home/daniel/DARM/darm_mujoco/darm_training/results/Test_DARMSF_DELTA_TARGET/SAC_darm_DarmSFHand-v0_a1dbe_00000_0_2023-02-11_16-49-13/results/Test_DARMSF_DELTA_TARGET/SAC_darm_DarmSFHand-v0_ad8de_00000_0_2023-02-11_16-56-43/checkpoint_000011)

(raylet) [2023-02-11 17:00:32,579 E 9254 9299] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_16-49-09_974082_9002 is over 95% full, available space: 1129005056; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-11 17:00:42,596 E 9254 9299] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_16-49-09_974082_9002 is over 95% full, available space: 1129254912; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-11 17:00:52,615 E 9254 9299] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_16-49-09_974082_9002 is over 95% full, available space: 1129238528; capacity: 31845081088. Object creation will fail if spilling is required.


In [11]:
# Get Algorithm from saved checkpoint
from ray.rllib.algorithms.algorithm import Algorithm
algo = Algorithm.from_checkpoint(best_checkpoint._local_path)
algo

2023-02-11 16:38:32,937	WARNING algorithm_config.py:488 -- Cannot create SACConfig from given `config_dict`! Property __stdout_file__ not supported.
2023-02-11 16:38:32,949	INFO algorithm.py:501 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(raylet) [2023-02-11 16:38:35,741 E 6815 6860] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_16-34-33_438228_6699 is over 95% full, available space: 1174347776; capacity: 31845081088. Object creation will fail if spilling is required.


(RolloutWorker pid=7501) Loaded XML file successfully


(RolloutWorker pid=7501) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=7501)   logger.warn(
(RolloutWorker pid=7500) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=7500)   logger.warn(
(RolloutWorker pid=7500) 2023-02-11 16:38:38,536	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


(RolloutWorker pid=7500) Loaded XML file successfully


(RolloutWorker pid=7502) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=7502)   logger.warn(


(RolloutWorker pid=7502) Loaded XML file successfully


/home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
2023-02-11 16:38:39,885	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2023-02-11 16:38:39,927	WARNING util.py:66 -- Install gputil for GPU system monitoring.


Loaded XML file successfully


SAC

In [11]:
# resume the interrupted run
tuner.fit()

2023-02-11 17:26:26,421	INFO trial_runner.py:688 -- A local experiment checkpoint was found and will be used to restore the previous experiment state.
2023-02-11 17:26:26,422	INFO trial_runner.py:825 -- Using following checkpoint to resume: /home/daniel/DARM/darm_mujoco/darm_training/results/Test_DARMSF_DELTA_TARGET/experiment_state-2023-02-11_17-23-28.json
2023-02-11 17:26:26,426	WARNING trial_runner.py:830 -- Attempting to resume experiment from /home/daniel/DARM/darm_mujoco/darm_training/results/Test_DARMSF_DELTA_TARGET. This will ignore any new changes to the specification.
2023-02-11 17:26:26,440	INFO tune.py:653 -- TrialRunner resumed, ignoring new add_experiment but updating trial resources.


(raylet) [2023-02-11 17:26:26,618 E 14732 14777] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_17-23-24_266872_14581 is over 95% full, available space: 1061683200; capacity: 31845081088. Object creation will fail if spilling is required.
wandb: Currently logged in as: danieladejumo. Use `wandb login --relogin` to force relogin
(SAC pid=15703) 2023-02-11 17:26:30,934	WARNING algorithm_config.py:488 -- Cannot create SACConfig from given `config_dict`! Property __stdout_file__ not supported.
(SAC pid=15703) 2023-02-11 17:26:31,413	INFO algorithm.py:501 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(raylet) [2023-02-11 17:26:36,627 E 14732 14777] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_17-23-24_266872_14581 is over 95% full, available space: 1061359616; capacity: 31845081088. Object creation will fail if spilling is required.


(RolloutWorker pid=15846) Loaded XML file successfully
(RolloutWorker pid=15844) Loaded XML file successfully


(RolloutWorker pid=15846) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=15846)   logger.warn(
(RolloutWorker pid=15844) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=15844)   logger.warn(
(RolloutWorker pid=15844) 2023-02-11 17:26:38,825	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


(RolloutWorker pid=15845) Loaded XML file successfully


(RolloutWorker pid=15845) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=15845)   logger.warn(
(SAC pid=15703) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(SAC pid=15703)   logger.warn(
(SAC pid=15703) 2023-02-11 17:26:40,232	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(SAC pid=15703) 2023-02-11 17:26:40,261	WARNING util.py:66 -- Install gputil for GPU system monitoring.


(SAC pid=15703) Loaded XML file successfully


(SAC pid=15703) 2023-02-11 17:26:40,442	INFO trainable.py:790 -- Restored on 192.168.152.36 from checkpoint: /tmp/checkpoint_tmp_7f50b6e15e2c473dba807bf1d398566d
(SAC pid=15703) 2023-02-11 17:26:40,442	INFO trainable.py:799 -- Current state after restoring: {'_iteration': 11, '_timesteps_total': None, '_time_total': 113.04964661598206, '_episodes_total': 114}
(SAC pid=15703) 2023-02-11 17:26:40,721	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to the add method results in the column being overwritten.
(SAC pid=15703) 2023-02-11 17:26:40,721	WARNING deprecation.py:47 -- DeprecationWarning: `concat_samples` has been deprecated. Use `concat_samples() from rllib.policy.sample_batch` instead. This will raise an error in the future!
(raylet) [2023-02-11 17:26:46,634 E 14732 14777] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_17-23-24_266872_14581 is over 95% full, avai

wandb: Adding directory to artifact (/home/daniel/DARM/darm_mujoco/darm_training/results/Test_DARMSF_DELTA_TARGET/SAC_darm_DarmSFHand-v0_6a944_00000_0_2023-02-11_17-23-28/checkpoint_000012)... Done. 0.0s
(raylet) [2023-02-11 17:27:36,665 E 14732 14777] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_17-23-24_266872_14581 is over 95% full, available space: 1055997952; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-11 17:27:46,672 E 14732 14777] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_17-23-24_266872_14581 is over 95% full, available space: 1055973376; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-11 17:27:56,678 E 14732 14777] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_17-23-24_266872_14581 is over 95% full, available space: 1055977472; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-11 17:2